In [ ]:
import jtrace
import os
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
HSC_fn = os.path.join(jtrace.datadir, "hsc", "HSC.yaml")
fiducial_telescope = jtrace.Telescope.makeFromYAML(HSC_fn)

In [ ]:
def rays(theta_x, theta_y, wavelength):
    return jtrace.parallelRays(z=1, outer=4.1, inner=0.0, 
                               theta_x=theta_x, theta_y=theta_y, wavelength=wavelength,
                               medium=jtrace.ConstMedium(1.0), nradii=40, naz=256)

In [ ]:
def perturb(PM_dx=0, PM_dy=0,
            cam_dx=0, cam_dy=0, cam_dz=0):
    telescope = fiducial_telescope
    if not (PM_dx == PM_dy == 0):
        telescope = telescope.withShift('PM', PM_dx, PM_dy, 0.0)
    if not (cam_dx == cam_dy == cam_dz == 0):
        for elt in ['G1', 'G1E', 'G2', 'G2E', 
                    'ADC1', 'ADC1E', 'ADC2', 'ADC2E',
                    'G3', 'G3E', 'G4', 'G4E',
                    'G5', 'G5E', 'F', 'FE', 'W', 'WE', 'D']:
            telescope = telescope.withShift(elt, cam_dx, cam_dy, cam_dz)
    return telescope

In [ ]:
@interact(wavelen=widgets.FloatSlider(min=300.0,max=1100.0,step=10.0,value=620.0),
          theta_x=widgets.FloatSlider(min=-0.75,max=0.75,step=0.01,value=-0.40),
          theta_y=widgets.FloatSlider(min=-0.75,max=0.75,step=0.01,value=0.0),
          PM_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.00),
          PM_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dz=widgets.FloatSlider(min=-10, max=10, step=0.01, value=0.0),
          logscale=widgets.FloatSlider(min=1.2, max=3, step=0.1, value=1.2))
def spot(wavelen, theta_x, theta_y, PM_dx, PM_dy, cam_dx, cam_dy, cam_dz, logscale):
    """Display a spot diagram for LSST.

    @param wavelen  Wavelength in nm
    @param theta_x  Field angle in degrees
    @param theta_y  Field angle in degrees
    @param PM_dx    Primary mirror x decenter in mm
    @param PM_dy    Primary mirror y decenter 
    @param cam_dx   Camera x decenter in mm
    @param cam_dy   Camera y decenter in mm
    @param cam_dz   Camera z despace in mm
    @param logscale Logarithmic axes zoom level
    """
    telescope = perturb(PM_dx*1e-3, PM_dy*1e-3, 
                        cam_dx*1e-3, cam_dy*1e-3, cam_dz*1e-3)
    rs, isecs = telescope.trace(rays(theta_x*np.pi/180, theta_y*np.pi/180, wavelen*1e-3))
    spots = [(isec.x0, isec.y0) for isec, ray in zip(isecs, rs) if not ray.isVignetted]
    spots = np.array(spots)
    spots -= np.mean(spots, axis=0)
    spots *= 1e6 # meters -> microns
    plt.figure(figsize=(4.5,4))
    plt.scatter(spots[:,0], spots[:,1], s=1, alpha=0.2)
    plt.xlim(-10**logscale, 10**logscale)
    plt.ylim(-10**logscale, 10**logscale)
    plt.title(r"$\theta_x = {:4.2f}\,,\theta_y = {:4.2f}$".format(theta_x, theta_y))
    plt.xlabel("microns")
    plt.ylabel("microns")

In [ ]:
#  http://stackoverflow.com/a/18968498
def planeFit(points):
    """
    p, n = planeFit(points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fit an d-dimensional plane to the points.
    Return a point, p, on the plane (the point-cloud centroid),
    and the normal, n.
    """
    from numpy.linalg import svd
    points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trialing dimensions
    ctr = points.mean(axis=1)
    x = points - ctr[:,np.newaxis]
    M = np.dot(x, x.T) # Could also use np.cov(x) here.
    return ctr, svd(M)[0][:,-1]

In [ ]:
@interact(wavelen=widgets.FloatSlider(min=300.0,max=1100.0,step=10.0,value=620.0),
          theta_x=widgets.FloatSlider(min=-0.75,max=0.75,step=0.01,value=-0.40),
          theta_y=widgets.FloatSlider(min=-0.75,max=0.75,step=0.01,value=0.0),
          PM_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.00),
          PM_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dx=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dy=widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.0),
          cam_dz=widgets.FloatSlider(min=-10, max=10, step=0.01, value=0.0),
          logscale=widgets.FloatSlider(min=-6, max=-4, step=0.1, value=-5.5))
def opd(wavelen, theta_x, theta_y, PM_dx, PM_dy, cam_dx, cam_dy, cam_dz, logscale):
    """Display optical path differences

    @param wavelen  Wavelength in nm
    @param theta_x  Field angle in degrees
    @param theta_y  Field angle in degrees
    @param PM_dx    Primary mirror x decenter in mm
    @param PM_dy    Primary mirror y decenter in mm
    @param cam_dx   Camera x decenter in mm
    @param cam_dy   Camera y decenter in mm
    @param cam_dz   Camera z despace in mm
    @param logscale Logarithmic colorbar zoom level
    """
    telescope = perturb(PM_dx*1e-3, PM_dy*1e-3, 
                        cam_dx*1e-3, cam_dy*1e-3, cam_dz*1e-3)
    rs, isecs = telescope.trace(rays(theta_x*np.pi/180, theta_y*np.pi/180, wavelen*1e-3))
    theta_opd = [(r.v.x, r.v.y, isec.t) for r, isec in zip(rs, isecs) if not r.isVignetted]
    theta_opd = np.array(theta_opd)
    opd = theta_opd[:,2]
    opd[:] -= np.mean(opd)
    x = theta_opd[:,0]
    y = theta_opd[:,1]
    p, n = planeFit(theta_opd[::10,:].T)
    const = np.dot(p, n)
    opd[:] -= (const-n[0]*x-n[1]*y)/n[2]
    plt.figure(figsize=(5.3,4))
    plt.scatter(x, y, c=opd, s=5, vmin=-10**logscale, vmax=10**logscale)
    plt.xlim(-0.4, 0.4)
    plt.ylim(-0.4, 0.4)
    plt.axhline(0.0, c='k')
    plt.axvline(0.0, c='k')
    plt.xlabel("vx")
    plt.ylabel("vy")
    plt.colorbar()